# Register to ARA

The aim is to register individual slices to the Allen Reference Atlas. Example solutions
for this type of registration are described here:
https://forum.image.sc/t/comparison-of-some-tools-for-brain-slice-to-atlas-registration/43260

We will start with ABBA. This require a stitched overview in a pyramidal format. 
We start by generating that

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# imports
from pathlib import Path
import numpy as np
from flexiznam.config import PARAMETERS
import iss_preprocess as iss
import matplotlib.pyplot as plt

In [ ]:
# define experiment to analysis
project = "becalia_rabies_barseq"
mouse = "BRYC65.1d"
chamber = "chamber_13"
ref_acq = "genes_round_1_1"

data_path = Path(project) / mouse / chamber
processed_path = Path(PARAMETERS["data_root"]["processed"])
registration_folder = processed_path / data_path / "register_to_ara"
registration_folder.mkdir(exist_ok=True)


## Stitch tiles

We will take all rois and stitch the `ops['ref_ch']` of  `reference_prefix` . The saved image are in the `register_to_ara` subfolder with a yaml for each to specify downsampling info. 

The same function will also generate two empty directories (if they don't exists yet): 
- `qupath_project`: to create and store a QuPath project needed to load data into ABBA
- `deepslice`: to store small downsampled images uploaded to deepslice webpage and the `results.xml`

In [ ]:
# for debug purpose
if False:
    iss.pipeline.ara_registration.overview_single_roi(
    data_path,
    roi=1,
    slice_id=1,)

In [ ]:
# Long step, generates all ome.tiff
# (16 minutes for 1 ch of the infamous chamber_13, maybe an hour for 4 channels
# with gaussian blur)
if False:
    # for sigma blur, we use about 2um of pixels
    pixel_size = 2
    sigma_blur = 5./pixel_size
    thumb = iss.pipeline.overview_for_ara_registration(
        data_path=data_path,
        sigma_blur=sigma_blur,
    )


# Using ABBA

The simplest is to do it from the virtual machine. See #ito-proj-znamenskiy-gpu-vm on
slack for connection information.


## Create a QuPath project

- Once logged in, start QuPath, open the `register_to_ara` folder in windows file explore and drag&Drop the `qupath_abba` folder in QuPath.
- Click Ok to create a new project
- Drag&Drop the `ome.tif` files that you want to use for registration
- Set Image Provider to "Bio-Formats builder", Set image type to "Fluorescence" and make sure that the tick box are unticked. Then import.
- Delete the image #2, these are the thumbnails and we don't need them. (select them, right click, remove image, delete associated data)
- Save the project and close QuPath


## Load into ABBA

- Start Fiji
- Type `ABBA` in the quick search bar and select `ABBA - ABBA Start`
- Select Allen Brain Atlas V3
- For saving disk space you can use the same folder for brain map data: `file:/C:/abba_atlases/mouse_brain_ccfv3.xml` and ontology data: `file:/C:/abba_atlases/1.json`
- Slicing coronal, BA: allen, then wait a bit.
- In `ABBA`, click Import > ABBA - Import QuPath Project. You can keep the default import settings
- Select the `project.qpproj` in `qupath_project`
- Set the initial axis position. V1 is around 9mm
- Set the slice spacing (usually 0.016mm if you slice front to back, -0.016 if you go the other way round)
- Look at https://biop.github.io/ijp-imagetoatlas/ for navigation/display info

## Register with DeepSlice
See: https://biop.github.io/ijp-imagetoatlas/registration_with_deepslice.html

- In ABBA, select all slices and select Align > DeepSlice (web)
- Select the deepslice folder to save downsample data and tick everything
- Wait, it takes time to downsample.
- Follow the prompt
- On the deepslice web page, tick everything
- Save the results.xml in the same deepslice folder

## Elastix

- In ABBA select all the slices
- In ABBA do first an affine transform
- Still in ABBA do the Elastix non rigid transform

## [Optional] Adjust manually

You can select bigwarp to adjust the registration 

## Generate coordinate images


To get the output, which is an image with 3 channels for the x,y and z coordinates of each
pixel in the ARA, click on export -> Image coordinates to imagej or something like that.

This will open a bunch of windows. Save them all in the `ara_coordinates` subfolder. To 
make that painless you can use the following imagej macro:

```
dir = getDirectory("Choose a Directory");
//ids=newArray(nImages);
for (i=0;i<nImages;i++) {
        selectImage(i+1);
        title = getTitle;
        print(title);
        //ids[i]=getImageID;

        saveAs("tiff", dir+title);
} 
run("Close All");
```

# Create ARA area ID map

Now we can use the coordinates to get a map of the area IDs

In [ ]:
# plot overlay
from iss_preprocess.vis import plot_registration
ops = np.load(processed_path / data_path / "ops.npy", allow_pickle=True).item()
figure_folder = processed_path / data_path / "figures"
REDO = False
if REDO:
    for roi in ops['use_rois']:
        fig = plot_registration(data_path, roi, reference_prefix="genes_round_1_1")
        fig.savefig(figure_folder / f"registration_roi{roi}.png", dpi=600)
    


# Get ARA info for spots

Finally we can get the ARA info just for the spots. We first need to get a dataframe of
spots in the reference coordinate.

In [ ]:
roi = ops["use_rois"][0]

shift_right, shift_down, tile_shape = iss.pipeline.register_adjacent_tiles(
    data_path, ref_coors=ops["ref_tile"], prefix="genes_round_1_1"
)
spots = iss.pipeline.merge_roi_spots(
    data_path, shift_right, shift_down, tile_shape, iroi=roi, prefix="genes_round"
)
iss.pipeline.ara_registration.spots_ara_infos(
    data_path, spots, roi, atlas_size=10, inplace=True
)
print(f"Spots in {len(spots['area_id'].unique())} areas.")
spots.head()


In [ ]:
# plot the spots
from cricksaw_analysis import atlas_utils
import bg_atlasapi as bga

area_map = iss.pipeline.ara_registration.make_area_image(data_path, roi, atlas_size=10)
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

add_borders = False  # add a flag because it's slow
if add_borders:
    reg_metadata = iss.pipeline.ara_registration.load_registration_reference_metadata(
        data_path, roi=roi
    )
    scale = 1 / reg_metadata["downsample_ratio"]
    atlas_utils.plot_borders_and_areas(
        ax,
        area_map,
        border_kwargs=dict(colors="black", alpha=0.6, linewidths=0.1, zorder=100),
    )
else:
    # no need to downsample
    scale = 1


atlas = bga.BrainGlobeAtlas("allen_mouse_100um")
labels = atlas.lookup_df
skip = 100
midline = spots.x.max() / 2
for area_id, df in spots.iloc[::skip].groupby("area_id"):
    ax.scatter(df.x.values * scale, df.y.values * scale, zorder=1, s=1)
    if area_id == 0:
        # Out of brain
        continue

    label = labels.loc[labels.id == area_id, "acronym"].iloc[0]
    left = df.loc[df.x < midline, ["x", "y"]].values * scale
    if len(left) < 10:
        continue
    centre = np.nanmean(left, axis=0)
    mi = np.nanmin(left, axis=0)
    ma = np.nanmax(left, axis=0)
    ar = np.sum((ma - mi) ** 2)
    if ar / scale > 10e6:
        ax.text(
            *centre, s=label, verticalalignment="center", horizontalalignment="center"
        )

ax.set_ylim(ax.get_ylim()[::-1])
fig.subplots_adjust(left=0, bottom=0, right=1, top=1)
leg = ax.legend(ncol=6, bbox_to_anchor=(1.04, 1), loc="upper left")
